In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/urban-noise-levels/urban_noise_levels.csv


In [2]:
import pandas as pd
import folium
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv("/kaggle/input/urban-noise-levels/urban_noise_levels.csv")
data.head()

,id,latitude,longitude,datetime,decibel_level,hour,day_of_week,is_weekend,temperature_c,humidity_%,...,population_density,park_proximity,industrial_zone,vehicle_count,honking_events,public_event,holiday,school_zone,noise_complaints,sensor_id
0,1,40.649816,-74.069147,2023-04-01 18:50:00,78.286408,18,5,1,16.710979,43.054747,...,28384,0,0,24,1,0,0,1,1,31
1,2,40.880286,-74.076511,2023-03-18 04:12:00,68.131845,4,5,1,10.088749,29.676246,...,24206,0,0,15,2,0,0,0,0,40
2,3,40.792798,-73.746873,2023-12-03 05:43:00,58.934966,5,6,1,19.911075,47.932832,...,1894,1,0,15,2,0,0,0,0,41
3,4,40.739463,-74.075227,2023-06-20 00:44:00,69.559042,0,1,0,3.477520,88.594010,...,2265,1,0,24,4,0,0,1,2,50
4,5,40.562407,-74.064025,2023-12-28 12:06:00,60.409097,12,3,0,17.356303,35.811699,...,23807,1,0,25,2,0,0,0,0,8


In [4]:
data = data[['latitude', 'longitude', 'decibel_level']]
data.isnull().sum()

latitude         0
longitude        0
decibel_level    0
dtype: int64

In [5]:
data['decibel_level'].describe()

count    2000.000000
mean       64.815110
std        10.069717
min        33.232962
25%        57.907870
50%        65.019535
75%        71.647588
max        97.430930
Name: decibel_level, dtype: float64

In [6]:
features = data[['decibel_level']]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)



In [7]:
# clustering
from sklearn.cluster import KMeans

kmeans = KMeans(
    n_clusters=3,
    random_state=42,
    n_init=10
)

data['cluster'] = kmeans.fit_predict(scaled_features)

In [8]:
data.groupby('cluster')['decibel_level'].mean()
# 0 --> rendah
# 1 --> sedang
# 2 --> tinggi

cluster
0    52.430353
1    77.194433
2    65.111144
Name: decibel_level, dtype: float64

In [9]:
m = folium.Map(
    location=[data.latitude.mean(), data.longitude.mean()],
    zoom_start=12
)
colors = {0: 'green', 1: 'orange', 2: 'red'}

for _, row in data.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=6,
        color=colors[row['cluster']],
        fill=True,
        fill_opacity=0.7,
        popup=f"""
        Noise Level: {row['decibel_level']} dB<br>
        Cluster: {row['cluster']}
        """
    ).add_to(m)

m